In [102]:
import pandas as pd
import numpy as np
df = pd.read_csv("adult-census-income.zip")
removal= df.isin(['?']).any()[df.isin(['?']).any()==True].index.tolist()
for col in removal:
        df[col]= df[col].replace("?", np.NaN)
        try:
            df[col] = df[col].astype(float)
            mean = (df[col].mean(skipna=True))
        except:
            mean = df[col].mode()
            mean = mean[0]
        df[col]= df[col].replace(np.NaN, mean)
df_clean = df
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df["income"] = le.fit_transform(df["income"])
X = df.drop(["income"],1)
y = df["income"]
num = X.select_dtypes(include = np.number)
cat = X.select_dtypes(exclude = np.number)
for x in list(cat):
    cat[x] = cat[x].str.lower()
import warnings
warnings.filterwarnings("ignore")
cat.drop(["education"],1,inplace = True)
cat = pd.get_dummies(cat, drop_first=True)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
num = pd.DataFrame(scaler.fit_transform(num), columns=list(num))

In [105]:
def remove_outlier(X,y,z):
    scaler = StandardScaler()
    scaled_X = pd.DataFrame(scaler.fit_transform(X), columns=list(X))
    for column_name in list(X):
        X = X.drop(scaled_X[scaled_X[column_name] <= -3].index)
        X = X.drop(scaled_X[scaled_X[column_name] >= 3].index)
        y = y.drop(scaled_X[scaled_X[column_name] <= -3].index)
        y = y.drop(scaled_X[scaled_X[column_name] >= 3].index)
        z = z.drop(scaled_X[scaled_X[column_name] <= -3].index)
        z = z.drop(scaled_X[scaled_X[column_name] >= 3].index)
        scaled_X = scaled_X.drop(scaled_X[scaled_X[column_name] <= -3].index)
        scaled_X = scaled_X.drop(scaled_X[scaled_X[column_name] >= 3].index)
        X = X.reset_index(drop=True)
        y = y.reset_index(drop=True)
        z = z.reset_index(drop=True)
        scaled_X = scaled_X.reset_index(drop=True)
    return(X,y,z)
num,y,cat = remove_outlier(num,y,cat)


In [109]:
y.shape

(29828,)

In [111]:
X = pd.concat([num,cat],1)
y = y

In [130]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import  GridSearchCV, train_test_split as tts
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_auc_score, roc_curve

In [126]:
log_reg = LogisticRegression(random_state=42, class_weight="balanced", C = 100)

In [127]:
log_reg.fit(X,y)
log_reg.score(X,y)

0.805551830494837

In [128]:
X_train, X_test, y_train, y_test = tts(X,y,test_size = 0.25, random_state = 42)
log_reg.fit(X_train,y_train)
y_pred = log_reg.predict(X_test)
print (accuracy_score(y_test,y_pred))
print (classification_report(y_test,y_pred))

0.8017969692905994
              precision    recall  f1-score   support

           0       0.95      0.79      0.86      5786
           1       0.54      0.84      0.66      1671

    accuracy                           0.80      7457
   macro avg       0.74      0.82      0.76      7457
weighted avg       0.85      0.80      0.81      7457



In [129]:
print (confusion_matrix(y_test,y_pred))

[[4569 1217]
 [ 261 1410]]


In [131]:
print (roc_auc_score(y_test,y_pred))

0.8167354060224612


In [135]:
log_reg = LogisticRegression(random_state=42, class_weight="balanced")

In [136]:
log_reg.fit(X,y)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=42, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [144]:
features = list(X)

In [145]:
fs = pd.DataFrame()
ranking = pd.Series(log_reg.coef_[0])

In [147]:
fs["features"] = features
fs["ranking"] = ranking

In [150]:
fs["ranking"] = np.abs(fs["ranking"])

In [157]:
fs = fs.sort_values(["ranking"], ascending=False)

In [160]:
features_1 = fs[:30]

In [161]:
from sklearn.feature_selection import SelectKBest, f_classif

In [162]:
skb = SelectKBest(score_func=f_classif, k = 10)

In [163]:
skb.fit(X,y)

SelectKBest(k=10, score_func=<function f_classif at 0x1a2dc65b90>)

In [170]:
fs

,features,ranking,pvalues
13,marital.status_married-af-spouse,2.497979,1.492557e-03
3,capital.gain,2.452599,0.000000e+00
14,marital.status_married-civ-spouse,1.989641,0.000000e+00
26,occupation_priv-house-serv,1.873840,1.223801e-09
12,workclass_without-pay,1.711998,4.491917e-02
...,...,...,...
1,fnlwgt,0.075098,1.178494e-01
76,native.country_taiwan,0.037656,6.027211e-02
18,marital.status_widowed,0.028456,4.777195e-26
31,occupation_transport-moving,0.027085,9.852607e-04


In [171]:
fs = pd.DataFrame()

In [172]:
fs["features"] = list(X)
fs["p_values"] = pd.Series(skb.pvalues_)

In [176]:
fs.sort_values(["p_values"], ascending = True)[:30]

,features,p_values
0,age,0.000000e+00
2,education.num,0.000000e+00
3,capital.gain,0.000000e+00
14,marital.status_married-civ-spouse,0.000000e+00
5,hours.per.week,0.000000e+00
34,relationship_own-child,0.000000e+00
16,marital.status_never-married,0.000000e+00
41,sex_male,2.850083e-294
21,occupation_exec-managerial,5.546128e-287
32,relationship_not-in-family,3.019114e-223


In [177]:
features_1

,features,ranking
13,marital.status_married-af-spouse,2.497979
3,capital.gain,2.452599
14,marital.status_married-civ-spouse,1.989641
26,occupation_priv-house-serv,1.873840
12,workclass_without-pay,1.711998
44,native.country_columbia,1.621579
36,relationship_wife,1.483319
46,native.country_dominican-republic,1.278011
68,native.country_outlying-us(guam-usvi-etc),1.029194
22,occupation_farming-fishing,0.992508
